## Test 3: Always Availabile

In [1]:
test_num = "test3"
member_to_chair_ratio = 5

save_optimal_group1_path = f"../data/{test_num}/group1_results.txt"
save_optimal_group2_path = f"../data/{test_num}/group2_results.txt"
save_visualizations_prefix_path = f"../data/{test_num}"
master_schedule_path = f"../data/{test_num}/master_schedule.png"

In [2]:
chairs_unavailability = [
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
]

members_unavailability = [
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
    [],
]

## Import Packages

In [3]:
!pip install gurobipy>=9.5.1

zsh:1: 9.5.1 not found


In [4]:
import gurobipy as gp
from gurobipy import GRB as GRB
import sys
sys.path.insert(0, "../utils")

import column_generation
import schedules
import unavailability
import utils_functions

## Column Generation

In [5]:
groups = unavailability.form_groups(chairs_unavailability, members_unavailability)
group1_unavailability = groups[0]
group2_unavailability = groups[1]

In [6]:
generic_columns = column_generation.generic_column_generation(member_to_chair_ratio)
group1_feasible_combinations = column_generation.group_column_generation(1, group1_unavailability, generic_columns)
group2_feasible_combinations = column_generation.group_column_generation(2, group2_unavailability, generic_columns)

Total possible combinations: 1048576
Total feasible combinations: 45046
Group 1 Unavailability Mapping: {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: [], 52: [], 53: [], 54: [], 55: [], 56: [], 57: [], 58: [], 59: [], 60: [], 61: [], 62: [], 63: [], 64: [], 65: [], 66: [], 67: [], 68: [], 69: [], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [], 77: [], 78: [], 79: [], 80: [], 81: [], 82: [], 83: [], 84: [], 85: [], 86: [], 87: [], 88: [], 89: [], 90: [], 91: [], 92: [], 93: [], 94: [], 95: [], 96: [], 97: [], 98: [], 99: [], 100: [], 101: [], 102: [], 103: [], 104: [], 105: [], 106: [], 107: [], 108: [], 109: [], 110: [], 111: [], 112:

## Group 1 Optimization Model

In [7]:
# Setup Gurobi key
# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '453f2da9-b8f2-448f-9f06-48081a9c3dc9')
e.setParam('WLSSECRET', 'd5675c81-7058-45f9-b6f2-30ca3855ed22')
e.setParam('LICENSEID', 868440)
e.start()

# Create the model within the Gurobi environment. This environment will be passed in as a parameter into all subsequent models
model = gp.Model(env=e)
model.setParam('Seed', 435)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868440
Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Set parameter Seed to value 435


In [8]:
# Sets
D = range(1,6)
T = range(1,171)
T_d = {d : (T[(d-1)*len(T)//len(D):(d)*len(T)//len(D)])for d in D}
C = range(1,6)
M = range(6,21)
I = (*C,*M)
H_t = {t : range(1,len(group1_feasible_combinations[t-1])) for t in T}

In [9]:
# penalties
# separated to make them easier to read
# taken from this graph: https://www.desmos.com/calculator/g5rhs6n2ke
penalties = [(0,0), (4,4), (8, 11.31), (12, 20.78), (16, 32), (34, 99.13)]
Q = range(1,len(penalties)+1)

In [10]:
# Parameters

# need to sort this one out with column generation

a = group1_feasible_combinations
f = [[sum(a[t-1][h-1]) // 6 for h in H_t[t]] for t in T]
n = 8 # Need to inpute
u = {q : penalties[q-1][1] for q in Q}
v = {q : penalties[q-1][0] for q in Q}
w = .001 # Need to inpute

In [11]:
indices = [(t, h) for t in T for h in H_t[t]]

In [12]:
# Decision Varbiables
alpha = model.addVars(indices, vtype=GRB.BINARY, name="alpha")
y = model.addVars(I, D, Q, lb=0.0, ub=1.0, vtype=GRB.CONTINUOUS, name = "y")

In [13]:
for i in I:
    for d in D:
        model.addSOS(GRB.SOS_TYPE2, [y[i,d,q] for q in Q])

In [14]:
# Objective Value
model.setObjective(sum(f[t-1][h-1]*alpha[t,h] for t in T for h in H_t[t]) - w * sum(u[q] * y[i,d,q] for i in I for d in D for q in Q),sense=GRB.MAXIMIZE)

In [15]:
# Constraints
model.addConstrs(sum(alpha[t,h] for h in H_t[t]) <= 1 for t in T)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) <= n for i in I for d in D)
model.addConstrs(sum(a[t-1][h-1][i-1]*alpha[t,h] for t in T_d[d] for h in H_t[t]) == sum(v[q]*y[i,d,q] for q in Q) for i in I for d in D)
model.addConstrs(sum(y[i,d,q] for q in Q) == 1 for i in I for d in D)

{(1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 5): <gurobi.Constr *Awaiting Model Update*>,
 (4, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 2): <gurobi.Constr *Awaiting Model Update*>,
 (4, 3): <gurobi.Constr *Awaiting Model Update*>,
 (4, 4): <gurobi.Constr *Awaiting Model Update*>,
 (4, 5): <gurobi.Constr *Awaiting Model Update*>,


In [16]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E214)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Academic license 868440 - for non-commercial use only - registered to es___@uwaterloo.ca
Optimize a model with 470 rows, 7658250 columns and 160807670 nonzeros
Model fingerprint: 0x3120b6ee
Model has 100 SOS constraints
Variable types: 600 continuous, 7657650 integer (7657650 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [4e-03, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Presolve removed 0 rows and 0 columns (presolve time = 3203s) ...
Presolve removed 0 rows and 0 columns (presolve time = 3292s) ...

Interrupt request received


In [ ]:
with open(save_optimal_group1_path, "w") as file:
  for t in T:
    for h in H_t[t]:
      if alpha[t,h].X == 1:
        file.write(f"{t} {a[t-1][h-1]}\n")

## Group 2 Optimization Model

In [ ]:
# Guaranteed same results from both models since no individual busy times.
with open(save_optimal_group2_path, "w") as file:
  for t in T:
    for h in H_t[t]:
      if alpha[t,h].X == 1:
        file.write(f"{t} {a[t-1][h-1]}\n")

## Save Schedule Results

In [ ]:
group1_optimal = utils_functions.read_optimal_txt(save_optimal_group1_path)
group2_optimal = utils_functions.read_optimal_txt(save_optimal_group2_path)

In [ ]:
group1_people = [*unavailability.group1_chairs, *unavailability.group1_members]
group2_people = [*unavailability.group2_chairs, *unavailability.group2_members]

schedules.save_group_schedules(group1_people, group1_optimal, group1_unavailability, save_visualizations_prefix_path)
schedules.save_group_schedules(group2_people, group2_optimal, group2_unavailability, save_visualizations_prefix_path)

In [ ]:
rooms_per_timeslot = schedules.map_timeslots_to_num_rooms(group1_optimal, group2_optimal)
schedules.display_master_schedule(rooms_per_timeslot, master_schedule_path)